In [ ]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [ ]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [ ]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar()
gui = MetalGUI(design)
hfss = design.renderers.hfss

In [ ]:
# Allow overwriting

design.overwrite_enabled = True

In [ ]:
otg1 = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='0um', pos_y='0.9mm',
orientation='180'))
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='-1.25mm',
orientation='270')) 

from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '125um']
#jogs[1] = ["L", '500um']
jogs[1] = ["R", '100um']
#jogs[3] = ["R", '125um']

rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='4 mm',
        hfss_wire_bonds = True,
        fillet='20 um',
        lead = Dict(
        start_straight='0.2mm',
        end_straight='0mm',
        end_jogged_extension=jogs),
        pin_inputs=Dict(
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground1', pin='open')), ))

gui.rebuild()
gui.autoscale()


In [ ]:
#Setup the launchpad location and orientation.
launch_options1 = dict(pos_x='-500um', pos_y='1050um', orientation='0')

lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options1)

#Setup the OpenToGround location and orientation.
launch_options2 = dict(pos_x='0.75mm',  pos_y='1.05mm', orientation='180')

lp2 = LaunchpadWirebondCoupled(design, 'P5_Q', options = launch_options2)

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options = Dict(
       # total_length='0.25 mm',
        #fillet='4 um',
      #  lead = dict(start_straight='0um'),
        pin_inputs=Dict(
            start_pin=Dict(component=lp.name, pin='tie'),
            end_pin=Dict(component=lp2.name, pin='tie')), )

meander = RouteStraight(design, 'feed',options=meander_options)
gui.rebuild()
gui.autoscale()

In [ ]:
# Screenshot the canvas only as a .png formatted file.
#gui.figure.savefig('shot.png')

#from IPython.display import Image, display
#_disp_ops = dict(width=500)
#display(Image('shot.png', **_disp_ops))

In [ ]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
hfss.connect_ansys()

In [ ]:
hfss.activate_eigenmode_design("Readout")

In [ ]:
hfss.render_design(['short_to_ground', 'readout', 'open_to_ground1', lp.name, 'feed', lp2.name], [])
hfss.save_screenshot()

In [ ]:
hfss.disconnect_ansys()